# LSTM을 사용한 고급 자동 텍스트 생성

In [1]:
from __future__ import print_function
import numpy as np
import random
import sys

In [2]:
path = "shakespeare_final.txt"
text = open(path).read().lower()
characters = sorted(list(set(text)))
print('corpus length:', len(text))
print('total chars:', len(characters))

corpus length: 581432
total chars: 61


In [3]:
char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 193798


In [5]:
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
        y[i, char2indices[next_chars[i]]] = 1

In [6]:
from keras.models import Sequential
from keras.layers import Dense, LSTM,Activation,Dropout
from keras.optimizers import RMSprop

In [7]:
model = Sequential()

model.add(LSTM(128, input_shape=(maxlen, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               97280     
_________________________________________________________________
dense (Dense)                (None, 61)                7869      
_________________________________________________________________
activation (Activation)      (None, 61)                0         
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [9]:
for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    for diversity in [0.2, 0.7,1.2]:
        print('\n----- diversity:', diversity)
        
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.
                
            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]
            
            generated += pred_char
            sentence = sentence[1:] + pred_char
            
            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\nOne combination completed \n")

----------------------------------------
Iteration 1
1515/1515 [==============================] - 90s 60ms/step - loss: 1.8906

----- diversity: 0.2
----- Generating with seed: "o please:
  my reason the physician to m"
o please:
  my reason the physician to my love the seart
    the seare the seent the laye the self in the laye
    the count the lay the seare the seart the liee a prolaie.
                                                                                                                                                                                                                                                                           
One combination completed 


----- diversity: 0.7
----- Generating with seed: "o please:
  my reason the physician to m"
o please:
  my reason the physician to man drange of
    if extune to benelies, for my fair, and that beal,
    what desick in what thase sies me, not what is not daich be
    who up the came thereaw it the say come.
 

  lefe if i have not your fear of your fears.
  orlando. and her grace.
  caesar. the wealth propliin. what the le blound that i have all look every
    the sillians as the world why the frong, and sake me
    carry the worn fear of it. use my lord,
    in the so do, of frome steeding o' the streach and there of sold
One combination completed 


----- diversity: 1.2
----- Generating with seed: "at, i pray you?
  enobarbus. a certain q"
at, i pray you?
  enobarbus. a certain quill or in;    
     where gave o manubucalieaed. o sladloft, in
  king. friends.
so he drames, madefy mild! i'll is; 
    than that fa, can not it own, and altey sempt.
  rosalind. he some of old's egedus.
  celia. nay, he reithing breath his -i'll, a ddame
    his loss before thee; oath fight, the lotl.
  orlando. man sce'gh the paint dispnequckens.
is him, come, iclippors!
  phebe. yan. now, we
One combination completed 

----------------------------------------
Iteration 6
1515/1515 [===========================

1515/1515 [==============================] - 78s 52ms/step - loss: 1.3187

----- diversity: 0.2
----- Generating with seed: "u gav'st it, else mistaking,
  so thy gr"
u gav'st it, else mistaking,
  so thy grace the presest that the pressesses the mander.
  cleopatra. why, the more that the mand have her forth,
    the man the mand the most brother that the most seem in the port.
  sink ond sirna. i have say the more that the pressesses and the prince.
    the prince that the man the servant that the part of the sure the part,
    when thou when the man the court of the servant the sweet but man
    t
One combination completed 


----- diversity: 0.7
----- Generating with seed: "u gav'st it, else mistaking,
  so thy gr"
u gav'st it, else mistaking,
  so thy greater fathers would thou a prornton and strange.
  antchies that would thou be with and regime.
    which i pudit this shame, but when your grace,
    and lack another letter of my loves of lander
    our son she must take the worl

  diana. what weigh this but he should i sails of the seven,
  and the leat fair; and thou thou have let them the severs.
    who grant, the contimvess, and ill and reward,
    no con for our for yourself.  not ho, whose thou seem is that thou shall be the queen; and 
One combination completed 


----- diversity: 1.2
----- Generating with seed: "ght, that i may change my mind,
  shall "
ght, that i may change my mind,
  shall dis'd in therefay in waulls i am agandon, and
    cle?buring own's?
  aires. gentous pleas!
  messenger. i will want her lofe cillest a good!
    i will let not seem slibry on pontenton's,
  but now good you roran come hrut kinkters.
  antony. playth strengthime ere it yost not he sick'd,
    thou have with wordsumet, and with the tell.
    he ary faith; of ingele outh
    have is seefured drof of
One combination completed 

----------------------------------------
Iteration 15
1515/1515 [==============================] - 83s 54ms/step - loss: 1.2831

----- divers

1515/1515 [==============================] - 90s 60ms/step - loss: 1.2592

----- diversity: 0.2
----- Generating with seed: "    as ever sigh'd upon a midnight pillo"
    as ever sigh'd upon a midnight pilloness,
    and shall the man is a for the court.
    the married the strong the duke and a father,
    and that the streamy to the sing; and the great strive the soldier,
    and we say the man is the breather to the stands that strike the service,
    and the man is the court the service to the streaming the court the court the many so strike the stream.
    i was the proud thou stat the string.
 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "    as ever sigh'd upon a midnight pillo"
    as ever sigh'd upon a midnight pillon on me.
  manler many so thy own revenger caesar,
  and lenonames her will doleinn this from fortune;
  and though that strike that know fell enswear.
    one with him who for.
                                  exit there wish the

    and he hath you new says and a turned death first,
  where i have not us i one him for not too little which is purition,
  all the tilless by but a man's duke's olivers you come
    our strong with entring speak between me, still,
    t
One combination completed 


----- diversity: 1.2
----- Generating with seed: " no wit by nature nor art may complain o"
 no wit by nature nor art may complain of the write na
    a drogns of starse pa; that are on't;
  and cotined thou like rest samess that sail
    most mect please prithes forsussice.
  parolles. why wise sit bowly in meld.
  enobarbus. i would you proves shall i'ls tal ids ambod oling;
    not thereif wisery find breat into now taicy and men
    serve to-night.
  oseters. hyspecop. no, midst you sinceful is a melds.
  celia. wh.
 ;    
One combination completed 

----------------------------------------
Iteration 24
1515/1515 [==============================] - 94s 62ms/step - loss: 1.2473

----- diversity: 0.2
----- Generating wi

<ipython-input-8-81a44e089a89>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / metric


"st i will speak to the speak the matter,
                      x[re, i arr
    show is the thing of this many with your self to the speak,
                       x['q!]!           
One combination completed 


----- diversity: 0.7
----- Generating with seed: "ine eye may be deceived.
    for fear of"
ine eye may be deceived.
    for fear of this dammsing days, i am
news would be my stood, i see all his mons
    were save be true; by the piter by the
    not have not to be offices that in a rimits to the end
           y[bqeobces, solemer and place
    o the great of the deards,
                    x'x
ent

                 she'e my sceller

  othusater and will be marriag fortuned in
    him; i am a turn of this virtue word,
    see
One combination completed 


----- diversity: 1.2
----- Generating with seed: "ine eye may be deceived.
    for fear of"
ine eye may be deceived.
    for fear of egepfuation on riseful or
    which horse grectious of faot nowe
    unswa'ne,
  who keepge in

    for         no ervur. the femcoy. no.  re

                  enter rosaliin
  countess. the flactimgess, to your gentrieh crust will
    the wincy! marrien by out but this put. thfeten ere i am,
    hate you o fe do's ; i do lave you that
    thrikerick to the confit the presentanto]
romei- is raight.
  egypt. it i'ntdest to tunly tragft yead.
  eros. 
  leis. upon youther empley shall 
One combination completed 

----------------------------------------
Iteration 29
1515/1515 [==============================] - 204s 135ms/step - loss: 1.2307

----- diversity: 0.2
----- Generating with seed: "endants





scene:
ephesus


<<this ele"
endants





scene:
ephesus


<<this electronic version of the second be to mark a dour of the duke's pain,
    and the most be the too letter to the count the courted of the duke's part,
    the dear that the will proped the duke's a maid so much a dead,
    the duke's pain the dear between to the count the duke's prost the strong to a man,
    and so 